## Install needed libraries


In [ ]:
!pip install transformers accelerate bitsandbytes ego4d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 13.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_

get the model

In [ ]:
# pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

access_token = ""

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", token=access_token)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", quantization_config=quantization_config, token=access_token)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:

import json
nar_data = json.load(open("/content/best_big_boy.json"))

In [ ]:
narrations = []

for vid in nar_data["videos"]:
    for clip in vid["clips"]:
        for ann in clip["annotations"]:
            for query in ann["language_queries"]:
                narrations.append(query.get("query", None))

print(narrations[0:5])

['#C C removes the brush on the paint box', '#C C paints a drawing book with the brush', '#C C mixes the paint on the paint box with the brush', '#C C touches the paint on the paint box with the brush', '#C C removes the brush on the paint box']


In [ ]:
print(len(narrations))

22038


In [ ]:
def iterate_in_batches(lst, batch_size=16):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

# Example usage:
my_list = [i for i in range(100)]  # Sample list
for batch in iterate_in_batches(my_list, 32):
    print(len(batch))

32
32
32
4


In [ ]:
import random

In [ ]:
random.seed(42)

In [ ]:
random_integers = [random.randint(0, len(narrations)) for _ in range(30)]

random_narrations = [narrations[i] for i in random_integers]

In [ ]:
questions = []
# we need better model
for batch in iterate_in_batches(random_narrations, 10):
  text=""
  for i in range(len(batch)):
    text+= f"{i+1}. " + batch[i] + "\n"


  input_text = f"""Given the following narrations describing the actions of a person, generate independent from one another {len(batch)} simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what , when and who questions only. Don't ask for feedback, you MUST only generate {len(batch)} questions.

******************************************
EXAMPLE:
Narrations:
1. #C C irons cloth on a table
2. #C C interacts with X

Corresponding Questions:
1. What does C iron?
2. Who does c interact with?
******************************************

Now it is your turn
Narrations:
{text}

Corresponding Questions:
"""
  input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

  # generated_text=""
  # while generated_text.count("?") < 2 + len(batch):
  outputs = model.generate(**input_ids, max_length=500, num_return_sequences=1, do_sample=False, top_k=50, top_p=0.95, temperature=0.7)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(generated_text)
  print("--------------------------------------------------------------------------------------")

  trimmed_text = generated_text[generated_text.rfind("1."):]
  trimmed_text = trimmed_text.strip()
  indexed_questions = trimmed_text.split("\n")
  for q in indexed_questions:
    questions.append(q[3:])

# print("\n\n\n ------------------------- Question Answer Pairs ----------------------------------")
# for q, a in zip(questions, random_narrations):
#   print(q, "->", a)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib

Given the following narrations describing the actions of a person, generate independent from one another 10 simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what, when and who questions only. Don't ask for feedback, you MUST only generate 10 questions.

******************************************
EXAMPLE:
Narrations:
1. #C C irons cloth on a table
2. #C C interacts with X

Corresponding Questions:
1. What does C iron?
2. Who does c interact with?
******************************************

Now it is your turn
Narrations:
1. #C C picks the snuggle plus
2. #C C sands a wood with a sander 
3. #C C inserts cable through a wood 
4. #C C moves around
5. #C C raises his left hand
6. #C C hums a song
7. #C C holds the part of the electric scooter with his left hand.
8. #C C holds the cloth
9. #O a lady X puts the cloth aside 
10. #C C takes the pot cover.


Corresponding Questions:
1. What does C pick?
2. What does C sand?
3. What does 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Given the following narrations describing the actions of a person, generate independent from one another 10 simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what, when and who questions only. Don't ask for feedback, you MUST only generate 10 questions.

******************************************
EXAMPLE:
Narrations:
1. #C C irons cloth on a table
2. #C C interacts with X

Corresponding Questions:
1. What does C iron?
2. Who does c interact with?
******************************************

Now it is your turn
Narrations:
1. #C C drops the bottle on the kitchen slab with her left hand
2. #C C picks a screw from the motorcycle part with his right hand
3. #C C walk around.
4. #C C drops the impact driver on the rack
5. #C C looks around 
6. #C C sticks a cellotape on the poster
7. #C C stretches the rubber band in her hand.
8. #C C Collects trash with leaf blower vacuum 
9. #C C puts the book in the shelf
10. #C C adjusts another r

In [ ]:
questions = []
# we need better model
for batch in iterate_in_batches(random_narrations, 10):
  text=""
  for i in range(len(batch)):
    text+= f"{i+1}. " + batch[i] + "\n"


  input_text = f"""Given the following narrations describing the actions of a person, generate independent from one another {len(batch)} simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what, when and who questions only.

Narrations:
{text}

Corresponding Questions:
"""
  input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

  # generated_text=""
  # while generated_text.count("?") < 2 + len(batch):
  outputs = model.generate(**input_ids, max_length=500, num_return_sequences=1, do_sample=False, top_k=50, top_p=0.95, temperature=0.7)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(generated_text)
  print("--------------------------------------------------------------------------------------")

  trimmed_text = generated_text[generated_text.rfind("1."):]
  trimmed_text = trimmed_text.strip()
  indexed_questions = trimmed_text.split("\n")
  for q in indexed_questions:
    questions.append(q[3:])

# print("\n\n\n ------------------------- Question Answer Pairs ----------------------------------")
# for q, a in zip(questions, random_narrations):
#   print(q, "->", a)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Given the following narrations describing the actions of a person, generate independent from one another 10 simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what, when and who questions only.

Narrations:
1. #C C picks the snuggle plus
2. #C C sands a wood with a sander 
3. #C C inserts cable through a wood 
4. #C C moves around
5. #C C raises his left hand
6. #C C hums a song
7. #C C holds the part of the electric scooter with his left hand.
8. #C C holds the cloth
9. #O a lady X puts the cloth aside 
10. #C C takes the pot cover.


Corresponding Questions:
1. Who picks the snuggle plus?
2. What does C pick?
3. When does C pick the snuggle plus?
4. Who sands a wood with a sander?
5. What does C sand?
6. When does C sand a wood with a sander?
7. Who inserts cable through a wood?
8. What does C insert?
9. When does C insert cable through a wood?
10. Who moves around?
11. What does C move around?
12. When does C move around?
13. 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Given the following narrations describing the actions of a person, generate independent from one another 10 simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what, when and who questions only.

Narrations:
1. #C C drops the bottle on the kitchen slab with her left hand
2. #C C picks a screw from the motorcycle part with his right hand
3. #C C walk around.
4. #C C drops the impact driver on the rack
5. #C C looks around 
6. #C C sticks a cellotape on the poster
7. #C C stretches the rubber band in her hand.
8. #C C Collects trash with leaf blower vacuum 
9. #C C puts the book in the shelf
10. #C C adjusts another rectangular wood on the floor


Corresponding Questions:
1. Who drops the bottle on the kitchen slab with her left hand?
2. Who picks a screw from the motorcycle part with his right hand?
3. Who walk around?
4. Who drops the impact driver on the rack?
5. Who looks around?
6. Who sticks a cellotape on the poster?
7. Who s

In [ ]:
questions = []
# we need better model
for batch in iterate_in_batches(random_narrations, 10):
  text=""
  for i in range(len(batch)):
    text+= f"{i+1}. " + batch[i] + "\n"


  input_text = f"""Given the following narrations describing the actions of a person, generate independent from one another {len(batch)} simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what, when questions only. Use different type of questions when possible. Avoid using sentences like 'What does C <verb>?'

Narrations:
{text}

Corresponding Questions:
"""
  input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

  # generated_text=""
  # while generated_text.count("?") < 2 + len(batch):
  outputs = model.generate(**input_ids, max_length=500, num_return_sequences=1, do_sample=False, top_k=50, top_p=0.95, temperature=0.7)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(generated_text)
  print("--------------------------------------------------------------------------------------")

  trimmed_text = generated_text[generated_text.rfind("1."):]
  trimmed_text = trimmed_text.strip()
  indexed_questions = trimmed_text.split("\n")
  for q in indexed_questions:
    questions.append(q[3:])

# print("\n\n\n ------------------------- Question Answer Pairs ----------------------------------")
# for q, a in zip(questions, random_narrations):
#   print(q, "->", a)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Given the following narrations describing the actions of a person, generate independent from one another 10 simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what, when questions only. Use different type of questions when possible. Avoid using sentences like 'What does C <verb>?'

Narrations:
1. #C C picks the snuggle plus
2. #C C sands a wood with a sander 
3. #C C inserts cable through a wood 
4. #C C moves around
5. #C C raises his left hand
6. #C C hums a song
7. #C C holds the part of the electric scooter with his left hand.
8. #C C holds the cloth
9. #O a lady X puts the cloth aside 
10. #C C takes the pot cover.


Corresponding Questions:
1. What does C pick?
2. What does C sand?
3. What does C insert?
4. What does C move?
5. What does C raise?
6. What does C hum?
7. What does C hold?
8. What does C take?
9. What does X put?
10. What does C take?

---------------------------------------------------------------------------

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Given the following narrations describing the actions of a person, generate independent from one another 10 simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what, when questions only. Use different type of questions when possible. Avoid using sentences like 'What does C <verb>?'

Narrations:
1. #C C drops the bottle on the kitchen slab with her left hand
2. #C C picks a screw from the motorcycle part with his right hand
3. #C C walk around.
4. #C C drops the impact driver on the rack
5. #C C looks around 
6. #C C sticks a cellotape on the poster
7. #C C stretches the rubber band in her hand.
8. #C C Collects trash with leaf blower vacuum 
9. #C C puts the book in the shelf
10. #C C adjusts another rectangular wood on the floor


Corresponding Questions:
1. What does C do with the bottle?
2. What does C do with the screw?
3. What does C do?
4. What does C do with the impact driver?
5. What does C do?
6. What does C do with the c

In [ ]:
questions = []
# we need better model
for batch in iterate_in_batches(random_narrations, 10):
  text=""
  for i in range(len(batch)):
    text+= f"{i+1}. " + batch[i] + "\n"


  input_text = f"""Given the following narrations describing the actions of a person, generate independent from one another {len(batch)} simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what and when questions only. Make the questions focused on the object and not on the verb.

Narrations:
{text}

Corresponding Questions:
"""
  input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

  # generated_text=""
  # while generated_text.count("?") < 2 + len(batch):
  outputs = model.generate(**input_ids, max_length=500, num_return_sequences=1, do_sample=False, top_k=50, top_p=0.95, temperature=0.7)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(generated_text)
  print("--------------------------------------------------------------------------------------")

  trimmed_text = generated_text[generated_text.rfind("1."):]
  trimmed_text = trimmed_text.strip()
  indexed_questions = trimmed_text.split("\n")
  for q in indexed_questions:
    questions.append(q[3:])

# print("\n\n\n ------------------------- Question Answer Pairs ----------------------------------")
# for q, a in zip(questions, random_narrations):
#   print(q, "->", a)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Given the following narrations describing the actions of a person, generate independent from one another 10 simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what and when questions only. Make the questions focused on the object and not on the verb.

Narrations:
1. #C C picks the snuggle plus
2. #C C sands a wood with a sander 
3. #C C inserts cable through a wood 
4. #C C moves around
5. #C C raises his left hand
6. #C C hums a song
7. #C C holds the part of the electric scooter with his left hand.
8. #C C holds the cloth
9. #O a lady X puts the cloth aside 
10. #C C takes the pot cover.


Corresponding Questions:
1. What does C pick?
2. What does C sand?
3. What does C insert?
4. What does C move?
5. What does C raise?
6. What does C hum?
7. What does C hold?
8. What does C take?
9. What does X put?
10. What does C take?

--------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Given the following narrations describing the actions of a person, generate independent from one another 10 simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what and when questions only. Make the questions focused on the object and not on the verb.

Narrations:
1. #C C drops the bottle on the kitchen slab with her left hand
2. #C C picks a screw from the motorcycle part with his right hand
3. #C C walk around.
4. #C C drops the impact driver on the rack
5. #C C looks around 
6. #C C sticks a cellotape on the poster
7. #C C stretches the rubber band in her hand.
8. #C C Collects trash with leaf blower vacuum 
9. #C C puts the book in the shelf
10. #C C adjusts another rectangular wood on the floor


Corresponding Questions:
1. What does C drop on the kitchen slab with her left hand?
2. What does C pick from the motorcycle part with his right hand?
3. What does C walk around?
4. What does C drop on the rack?
5. What does C look a

tutors example

In [ ]:
tutor_text = """Given the following narrations describing the actions of a person, generate a set of simple queries (one per line) that could be answered by looking at the video segments corresponding to these narrations:
the person waters the plants using a water gun
the person touches the plant with the left hand
the person waters the plants using a water gun
the person stares at the plant
the person touches the soil with the left hand
"""

input_ids = tokenizer(tutor_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_length=500)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Given the following narrations describing the actions of a person, generate a set of simple queries (one per line) that could be answered by looking at the video segments corresponding to these narrations:
the person waters the plants using a water gun
the person touches the plant with the left hand
the person waters the plants using a water gun
the person stares at the plant
the person touches the soil with the left hand
the person stares at the plant
the person touches the plant with the right hand
the person touches the soil with the right hand
the person touches the plant with the right hand
the person touches the soil with the right hand
the person touches the plant with the left hand
the person touches the soil with the left hand
the person touches the plant with the left hand
the person touches the soil with the left hand
the person touches the plant with the left hand
the person touches the soil with the left hand
the person touches the plant with the left hand
the person touch

In [ ]:
print(len(narrations))

22038


In [ ]:
# import csv

# # Specify the file name
# file_name = "narrations.csv"

# # Open the file in write mode
# with open(file_name, mode='w', newline='') as file:
#     writer = csv.writer(file)

#     # Write each string as a separate row
#     for string in narrations:
#         writer.writerow([string])


In [ ]:
# import pandas as pd

# # Specify the file name
# file_name = "/content/narrations.csv"

# # Read the CSV file into a DataFrame
# df = pd.read_csv(file_name, header=None)

# # Print the first few rows of the DataFrame
# print(df.head())


                                                   0
0            #C C removes the brush on the paint box
1          #C C paints a drawing book with the brush
2  #C C mixes the paint on the paint box with the...
3  #C C touches the paint on the paint box with t...
4            #C C removes the brush on the paint box
